In [1]:
from text_decomposition_prompt import text_decomposition_prompt
print(text_decomposition_prompt("TEST"))


--Goal--
Given a text, segment it into multiple semantic units, each containing detailed descriptions of specific events or activities.
Perform the following tasks:
--Steps--
1. Provide a summary for each semantic unit while retaining all crucial details relevant to the original context.
2. Extract all entities directly from the original text of each semantic unit, not from the paraphrased. Format each entity name in UPPERCASE. You should extract all entities including times, locations, people, organizations and all kinds of entities.
3. From the entities extracted in Step 2, list all relationships within the semantic unit and the corresponding original context in the form of string separated by comma: "ENTITY_A, RELATION_TYPE, ENTITY_B". The RELATION_TYPE could be a descriptive sentence, while the entities involved in the relationship must come from the entity names extracted in Step 2. Please make sure the string contains three elements representing two entities and the relationship 

In [2]:
with open(r"C:\Users\HP\Desktop\Projects\NodeRAG\API_KEY.txt", "r", encoding="utf-8") as f:
    API_KEY = f.read()

In [3]:
import pandas as pd
medical_chunks_path = r"C:\Users\HP\Desktop\Projects\NodeRAG\chunking\medical_chunks.parquet"

In [4]:
medical_chunks = pd.read_parquet(medical_chunks_path)
medical_chunks

,chunk_id,corpus_name,chunk,chunk_length
0,0,Medical,About basal cell skin cancer What is basal cel...,399
1,1,Medical,Hypodermis – The deepest tissue layer is made ...,376
2,2,Medical,Causes and risk factors Basal cell skin cancer...,397
3,3,Medical,Recurrence is when basal cell skin cancer retu...,387
4,4,Medical,2 Testing for basal cell skin cancer 10 Genera...,105
...,...,...,...,...
549,549,Medical,Is there a portal where I can get copies of my...,398
550,550,Medical,"Is home care after treatment needed? If yes, w...",399
551,551,Medical,What will you target? What if I’ve already had...,393
552,552,Medical,What are the chances the cancer will return or...,399


In [5]:
from google import genai

def call_gemini(text):
    client = genai.Client(api_key = API_KEY)
    response = client.models.generate_content(
        model="gemini-2.5-flash-lite",
        contents=text_decomposition_prompt(text),
    )
    return response.text

In [6]:
import os
path = "medical_responses.parquet"
if os.path.exists(path):
    df = pd.read_parquet(path)
    responses = dict(zip(df["index"], df["response"]))
else:
    responses = {}

responses

{0: '[\n{\n"semantic_unit": "Basal cell skin cancer, also known as basal cell carcinoma (BCC), is the most common form of skin cancer, with approximately 3 million cases diagnosed annually in the United States. It is typically curable through surgery to remove the cancerous cells.",\n"entities": [\n"BASAL CELL SKIN CANCER",\n"BASAL CELL CARCINOMA (BCC)",\n"3 MILLION CASES",\n"UNITED STATES",\n"SURGERY"\n],\n"relationships": [\n"BASAL CELL SKIN CANCER, also known as, BASAL CELL CARCINOMA (BCC)",\n"BASAL CELL SKIN CANCER, diagnosed in, UNITED STATES",\n"BASAL CELL SKIN CANCER, treated with, SURGERY"\n]\n},\n{\n"semantic_unit": "Basal cell skin cancer originates when basal cells in the epidermis become abnormal and grow uncontrollably. This cancer commonly appears on sun-exposed areas like the face, head, neck, arms, legs, and trunk, but can manifest anywhere on the body.",\n"entities": [\n"BASAL CELL SKIN CANCER",\n"BASAL CELLS",\n"EPIDERMIS",\n"FACE",\n"HEAD",\n"NECK",\n"ARMS",\n"LEGS",

In [7]:
stop here

SyntaxError: invalid syntax (4067800170.py, line 1)

In [ ]:
from tqdm import tqdm

start_idx = 0
end_idx = len(medical_chunks)
for idx, row in tqdm(medical_chunks.iloc[start_idx:end_idx].iterrows()):
    chunk_text = row['chunk']
    value = call_gemini(chunk_text)
    key = idx
    responses[key] = value
    if idx % 10 == 0:
        print("Key:", key)
        print('-'*40)

In [ ]:
stop here

In [12]:
from tqdm import tqdm
idx_list = sorted([450, 431, 512, 439])
for idx in tqdm(idx_list):
    row = medical_chunks.loc[idx]
    chunk_text = row['chunk']
    value = call_gemini(chunk_text)
    key = idx
    responses[key] = value
    print("Key:", key)

 25%|██▌       | 1/4 [00:08<00:25,  8.66s/it]

Key: 431


 50%|█████     | 2/4 [00:21<00:21, 11.00s/it]

Key: 439


 75%|███████▌  | 3/4 [00:32<00:11, 11.13s/it]

Key: 450


100%|██████████| 4/4 [00:45<00:00, 11.48s/it]

Key: 512


In [ ]:
stop here

In [13]:
df = pd.DataFrame(list(responses.items()), columns=["index", "response"])
df.to_parquet("medical_responses.parquet", index=False)